In [17]:
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import randint

In [18]:
dataset = pd.read_csv(r"F:\cache_data\frequency_filter\dy\scaler_csv\dy_stander_filter_havelabel.csv")
dataset.dropna(inplace=True)

In [21]:
dataset = dataset[dataset['label'] == 'fish_net']

In [25]:
dataset= dataset.drop(columns=['label'])

In [26]:
dataset

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL,TZ_label
0,13d,1021.6,0.186124,5.645218,940.00836,81.591610,0.000000,4.197873,13.658210,0.010529,...,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7,34
1,13d,701.7,1.216353,2.768598,694.73010,6.969910,0.000000,-1.750685,9.647439,-0.005612,...,16.391666,1.587500,1.583333,191.06468,472945.0,2907200.0,6.843954,4,3,34
2,13d,661.0,0.785157,5.674496,663.14667,0.000000,2.146667,1.533146,0.001766,0.000064,...,16.375000,1.588333,1.583333,191.17732,473845.0,2907200.0,6.713590,4,39,34
3,13d,869.8,0.820064,3.988137,857.71760,12.082397,0.000000,2.137851,13.153590,0.005155,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861600.0,5.598476,6,3,34
4,13d,1031.5,0.443475,4.917029,903.36255,128.137450,0.000000,0.071758,14.658850,0.000746,...,15.766666,1.555833,1.658333,194.86105,443245.0,2861900.0,5.441568,3,7,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,1e,1646.3,1.134341,1.592065,1615.18730,31.112793,0.000000,1.626131,14.525656,0.003527,...,12.325000,1.260000,2.100000,192.85692,434245.0,2918000.0,5.110536,3,7,47
4388,1e,1600.5,0.950942,1.269871,1560.65940,39.840576,0.000000,-0.021284,9.449365,0.000551,...,12.758333,1.300000,2.000000,191.90393,434545.0,2918000.0,5.209802,3,7,47
4389,1e,1656.4,0.719550,0.394788,1613.64420,42.755860,0.000000,-2.737058,22.095870,-0.005495,...,12.483334,1.274167,2.066667,192.62935,433345.0,2918300.0,5.140053,3,7,47
4390,1e,1630.9,0.629897,0.141897,1615.21340,15.686645,0.000000,-0.026897,13.455787,-0.000136,...,12.325000,1.260000,2.100000,192.85692,433645.0,2918300.0,5.111474,3,7,47


In [27]:
# 标签化
dataset["TZ_label"] = dataset.TZ.astype("category").cat.codes
result = dataset.groupby('TZ_label')["TZ"].apply(lambda x: list(x.unique())).to_dict()
print(result)

{0: ['101'], 1: ['102'], 2: ['103'], 3: ['104'], 4: ['105'], 5: ['106'], 6: ['107'], 7: ['108'], 8: ['112'], 9: ['113'], 10: ['114'], 11: ['115'], 12: ['116'], 13: ['117'], 14: ['118'], 15: ['119'], 16: ['11e'], 17: ['120'], 18: ['125'], 19: ['126'], 20: ['129'], 21: ['12b'], 22: ['12c'], 23: ['12d'], 24: ['12e'], 25: ['12f'], 26: ['130'], 27: ['131'], 28: ['132'], 29: ['135'], 30: ['136'], 31: ['137'], 32: ['138'], 33: ['139'], 34: ['13d'], 35: ['13e'], 36: ['13f'], 37: ['141'], 38: ['143'], 39: ['14d'], 40: ['14e'], 41: ['14f'], 42: ['15'], 43: ['15d'], 44: ['15e'], 45: ['15f'], 46: ['18d'], 47: ['1e'], 48: ['1f'], 49: ['21'], 50: ['22'], 51: ['23'], 52: ['24'], 53: ['26'], 54: ['29'], 55: ['2b'], 56: ['2f'], 57: ['30'], 58: ['31'], 59: ['32'], 60: ['33'], 61: ['34'], 62: ['4d'], 63: ['4e'], 64: ['4f'], 65: ['61d'], 66: ['61e'], 67: ['61f'], 68: ['63'], 69: ['63b'], 70: ['63d'], 71: ['63e'], 72: ['63f'], 73: ['64d'], 74: ['64f'], 75: ['65'], 76: ['65d'], 77: ['65e'], 78: ['67'], 79: 

In [28]:
dataset['DZ']=dataset['DZ'].astype('category')
dataset['DL']=dataset['DL'].astype('category')
dataset['TZ_label']=dataset['TZ_label'].astype('category')

In [29]:
features = dataset.columns[1:-1]
target = "TZ_label"

In [30]:
# 计算每个类别的样本数
class_counts = dataset['TZ_label'].value_counts()

In [31]:
# 划分数据集
X = dataset[features]
y = dataset[target]
# 分割训练集和测试集
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# 定义参数分布进行随机搜索
param_dist = {
    'n_estimators': randint(100, 500),  # 树的数量
    'max_features': ['auto', 'sqrt', 'log2'],  # 最大特征数
    'max_depth': randint(10, 100),  # 最大深度
    'min_samples_split': randint(2, 11)  # 分割的最小样本数
}

# 创建Balanced Random Forest分类器实例
brf = BalancedRandomForestClassifier(random_state=42)

# 创建RandomizedSearchCV实例
random_search = RandomizedSearchCV(brf, param_distributions=param_dist, n_iter=100, cv=3, 
                                   scoring='accuracy', verbose=2, random_state=42, n_jobs=-1)

# 拟合RandomizedSearchCV
random_search.fit(Xtrain, ytrain)

# 打印最佳参数
print("Best parameters found: ", random_search.best_params_)

# 使用最佳参数的模型进行预测
y_pred = random_search.predict(Xtest)

# 打印分类报告
print(classification_report(ytest, y_pred))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
99 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
52 fits failed with the following error:
Traceback (most recent call last):
  File "d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\worker_code\.venvgis\lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "d:\w

Best parameters found:  {'max_depth': 98, 'max_features': 'log2', 'min_samples_split': 2, 'n_estimators': 484}
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         8
          10       0.05      1.00      0.09         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.08      0.50      0.13         2
          14       0.00      0.00      0.00         3
          15       0.00 

d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site

In [13]:
# 定义参数网格进行网格搜索
param_grid = {
    'n_estimators': list(range(1,500,100)),  # 树的数量
    'max_features': ['auto', 'sqrt', 'log2'],  # 最大特征数
    'max_depth': list(range(1,200,50)),  # 最大深度
    'min_samples_split': list(range(1,10,2))  # 分割的最小样本数
}

# 创建Balanced Random Forest分类器实例
brf = BalancedRandomForestClassifier(random_state=42)

# 创建GridSearchCV实例
grid_search = GridSearchCV(estimator=brf, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# 拟合GridSearchCV
grid_search.fit(Xtrain, ytrain)

# 打印最佳参数
print("Best parameters found: ", grid_search.best_params_)
# 使用最佳参数的模型进行预测
y_pred = grid_search.predict(Xtest)

# 打印分类报告
print(classification_report(ytest, y_pred))

Fitting 3 folds for each of 300 candidates, totalling 900 fits


d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
420 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
198 fits failed with the following error:
Traceback (most recent call last):
  File "d:\worker_code\.venvgis\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\worker_code\.venvgis\lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "d:

Best parameters found:  {'max_depth': 51, 'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 401}
              precision    recall  f1-score   support

           0       0.25      0.33      0.29         3
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4
           4       0.12      1.00      0.22         1
           5       0.00      0.00      0.00        18
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00        42
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00        18
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         3
          12       1.00      0.17      0.29         6
          13       0.09      1.00      0.17         1
          14       0.00      0.00      0.00         4
          15       0.00 

d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site

In [11]:

# 创建Balanced Random Forest分类器实例
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=42)

# 使用原始的训练数据训练模型
brf.fit(Xtrain, ytrain)

# 进行预测
y_pred = brf.predict(Xtest)

# 评估模型
print(f"Accuracy: {accuracy_score(ytest, y_pred)}")
print(classification_report(ytest, y_pred))


d:\worker_code\.venvgis\lib\site-packages\imblearn\ensemble\_forest.py:576: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
d:\worker_code\.venvgis\lib\site-packages\imblearn\ensemble\_forest.py:588: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
d:\worker_code\.venvgis\lib\site-packages\imblearn\ensemble\_forest.py:600: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.


Accuracy: 0.05823627287853577
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4
           4       0.07      1.00      0.12         1
           5       0.00      0.00      0.00        18
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00        42
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00        18
          10       0.00      0.00      0.00         4
          11       0.11      0.33      0.17         3
          12       0.14      0.17      0.15         6
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         4
          15       0.11      0.12      0.12         8
          16       0.00      0.00      0.00        

d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
d:\worker_code\.venvgis\lib\site